In [1]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import tikzplotlib

import json
from datetime import datetime

In [2]:
projects_df = pd.read_csv('/root/data/projects.csv',
                         parse_dates=['project_created_at', 'project_last_pushed_at', 'project_updated_at'])

packages_df = pd.read_csv('/root/data/packages_0_499.csv')
geiger_df = pd.read_csv('/root/data/geiger/geiger_findings_0_499.csv')

vet_df = pd.read_csv('/root/data/linters/vet_findings_0_499.csv')
gosec_df = pd.read_csv('/root/data/linters/gosec_findings_0_499.csv')

grep_df = pd.read_csv('/root/data/linters/grep_findings_0_499.csv')
ast_df = pd.read_csv('/root/data/ast/ast_findings_0_499.csv')
function_df = pd.read_csv('/root/data/ast/functions_0_499.csv')
statement_df = pd.read_csv('/root/data/ast/statements_0_499.csv')

# Figures included in the thesis document